In [3]:
import json
from IPython.display import display, HTML
from datetime import datetime, timedelta
from bs4 import BeautifulSoup

# Load the data from the file
with open('rounded_structured_metal_data.json', 'r') as file:
    data = json.load(file)
    metals_data = data["Metals"]

# Define periods for price changes
periods = {
    "One Day": 1,
    "One Week": 7,
    "One Month": 30,
    "Three Months": 90,
    "One Year": 365
}

# Process data to get percentage and numerical changes
dashboard_data = []
for metal in metals_data:
    metal_name = metal["Metal_Name"]
    unit = metal["Unit"]
    prices = metal["Prices"]
    
    # Sort prices by date
    prices = sorted(prices, key=lambda x: datetime.strptime(x["Date_Collected"], '%Y-%m-%d'), reverse=True)
    
    current_price = prices[0]["Price"]
    price_changes = {}
    current_date = datetime.strptime(prices[0]["Date_Collected"], '%Y-%m-%d')
    
    for label, days in periods.items():
        target_date = current_date - timedelta(days=days)
        
        # Find price closest to target date
        price_on_target_date = next((p["Price"] for p in prices if datetime.strptime(p["Date_Collected"], '%Y-%m-%d') <= target_date), None)
        
        if price_on_target_date:
            change_value = current_price - price_on_target_date
            percent_change = (change_value / price_on_target_date) * 100
            price_changes[label] = {
                "Numerical": round(change_value, 2),
                "Percentage": f"{round(percent_change, 2)}%"
            }
        else:
            price_changes[label] = {
                "Numerical": "-",
                "Percentage": "-"
            }
    
    dashboard_data.append({
        "Metal Name": metal_name,
        "Unit": unit,
        "Current Price": current_price,
        "Price Changes": price_changes
    })

def create_dashboard_table(data):
    # Start HTML table
    html = ["<table width='100%'><tr>"]
    headers = ["Metal Name", "Unit", "Current Price (AUD)"]
    for period in ["One Day", "One Week", "One Month", "Three Months", "One Year"]:
        headers.append(f"{period} Change (%)")  # Only percentage headers
    html.append("".join([f"<th>{header}</th>" for header in headers]))
    html.append("</tr>")
    
    for metal_data in data:
        html.append("<tr>")
        html.append(f"<td>{metal_data['Metal Name']}</td><td>{metal_data['Unit']}</td><td>${metal_data['Current Price']:,.2f}</td>")  # Format as dollar
        for period_change in metal_data["Price Changes"].values():
            percentage_display = period_change['Percentage']
            html.append(f"<td>{percentage_display}</td>")  # Only percentage data
        html.append("</tr>")
    
    html.append("</table>")
    return ''.join(html)

new_table_html = create_dashboard_table(dashboard_data)

# Now, instead of writing the new_table_html directly to the file, 
# use BeautifulSoup to update only the specific div with id="content".

# Read the existing HTML file
with open('metal_price_display.html', 'r') as file:
    html_content = file.read()

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Locate the div with id 'content'
content_div = soup.find('div', {'id': 'content'})

# Check if the div is found
if content_div:
    # Replace the contents of the div with the new table HTML
    content_div.clear()  # Clear the existing content
    content_div.append(BeautifulSoup(new_table_html, 'html.parser'))  # Insert the new table

    # Write the updated HTML back to the file
    with open('metal_dashboard.html', 'w') as file:
        file.write(str(soup.prettify()))  # Write the pretty-printed HTML content
else:
    print('The div with id="content" was not found in the HTML file.')